In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dense, Reshape,  LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

import random

In [ ]:
def load_dataset(dataset_folder):
    X, y = [], []
    class_labels = os.listdir(dataset_folder)
    
    for label, class_folder in enumerate(class_labels):
        class_path = os.path.join(dataset_folder, class_folder)
        mfcc_files = os.listdir(class_path)
        
        for mfcc_file in mfcc_files:
            file_path = os.path.join(class_path, mfcc_file)
            mfcc_data = np.load(file_path)
            X.append(mfcc_data)
            y.append(label)
    
    return np.array(X), np.array(y)

In [ ]:
dataset_folder = "E:\\UGRP\\npy\\mfcc"
#dataset_folder = "E:\\UGRP\\npy\\sg"
#dataset_folder = "E:\\UGRP\\npy\\cg"

X, y = load_dataset(dataset_folder)

In [ ]:
def create_siamese_network(input_shape):
    input_layer = Input(shape=input_shape)
    reshaped_input = Reshape((2498, 13))(input_layer)
    
    x = LSTM(128, return_sequences=True)(input_layer)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    embedding = Dense(64, activation='sigmoid')(x)

    return Model(input_layer, embedding)

In [ ]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = K.sum(K.square(anchor - positive), axis=-1)
    neg_dist = K.sum(K.square(anchor - negative), axis=-1)
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)
    return loss

In [ ]:
input_shape = (13, 2498)
siamese_network = create_siamese_network(input_shape)
siamese_network.compile(loss=triplet_loss, optimizer=Adam(0.0001))
siamese_network.summary()

In [ ]:
def generate_triplets(X, y, num_triplets):
    triplets = []
    classes = np.unique(y)
    
    for _ in range(num_triplets):
        anchor_class = random.choice(classes)
        negative_class = random.choice([cls for cls in classes if cls != anchor_class])

        anchor_idx = random.choice(np.where(y == anchor_class)[0])
        positive_idx = random.choice(np.where(y == anchor_class)[0])
        negative_idx = random.choice(np.where(y == negative_class)[0])

        anchor = X[anchor_idx]
        positive = X[positive_idx]
        negative = X[negative_idx]

        if anchor.ndim < 3:
            anchor = np.expand_dims(anchor, axis=0)
        if positive.ndim < 3:
            positive = np.expand_dims(positive, axis=0)
        if negative.ndim < 3:
            negative = np.expand_dims(negative, axis=0)

        triplets.append([anchor, positive, negative])

    return np.array(triplets)

In [ ]:
import random 
num_triplets = 10000000  
triplets = generate_triplets(X, y, num_triplets)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

num_epochs = 10
batch_size = 32

for epoch in range(num_epochs):
    losses = []
    for i in range(0, len(triplets), batch_size):
        batch_triplets = triplets[i:i + batch_size]
        anchor, positive, negative = zip(*batch_triplets)

        with tf.GradientTape() as tape:
            anchor_embeddings = siamese_network(anchor)
            positive_embeddings = siamese_network(positive)
            negative_embeddings = siamese_network(negative)

            loss = triplet_loss(None, [anchor_embeddings, positive_embeddings, negative_embeddings])

        gradients = tape.gradient(loss, siamese_network.trainable_variables)
        optimizer.apply_gradients(zip(gradients, siamese_network.trainable_variables))
        losses.append(loss.numpy())

    print(f"Epoch {epoch + 1}, Loss: {np.mean(losses)}")

In [ ]:
def euclidean_distance(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

In [ ]:
vector_a = np.random.normal(0, 1, (13, 2498))
vector_b = -vector_a

vector_a = vector_a.reshape(1, 13, -1)
vector_b = vector_b.reshape(1, 13, -1)

embedding1 = siamese_network.predict([vector_a, vector_b])
embedding2 = siamese_network.predict([vector_a, vector_a])

min_score = euclidean_distance(embedding1, embedding1)
max_score = euclidean_distance(embedding1, embedding2)

In [ ]:
def normalize_value(x, min_val=max_score, max_val=min_score):
    return (x - min_val) / (max_val - min_val)

In [ ]:
def result(vects):
    embedding1 = siamese_network.predict(vects[0])
    embedding2 = siamese_network.predict(vects[1])
    distance = euclidean_distance(embedding1, embedding2)
    return normalize_value(distance)

In [ ]:
mfcc_vector1 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\1.1 표절_29s_mfcc.npy')
mfcc_vector2 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\1.1 표절 key 3_29s_mfcc.npy')
mfcc_vector3 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\1.1 표절 key -3_29s_mfcc.npy')
mfcc_vector4 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\1.1 표절 noise 1_29s_mfcc.npy')
mfcc_vector5 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\1.1 표절 noise 2_29s_mfcc.npy')
mfcc_vector6 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\1.2 표절_29s_mfcc.npy')

mfcc_vector7 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\2.1 표절_29s_mfcc.npy')
mfcc_vector8 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\2.1 표절 key -3_29s_mfcc.npy')
mfcc_vector9 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\2.1 표절_29s_mfcc.npy')
mfcc_vector10 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\2.1 표절 key -3_29s_mfcc.npy')
mfcc_vector11 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\2.1 표절_29s_mfcc.npy')
mfcc_vector12 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\2.2 표절_29s_mfcc.npy')

mfcc_vector13 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\3.1 커버_29s_mfcc.npy')
mfcc_vector14 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\3.1 커버 key 3_29s_mfcc.npy')
mfcc_vector15 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\3.1 커버 key -3_29s_mfcc.npy')
mfcc_vector16 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\3.1 커버 noise 1_29s_mfcc.npy')
mfcc_vector17 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\3.1 커버 noise 2_29s_mfcc.npy')
mfcc_vector18 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\3.2 커버_29s_mfcc.npy')

mfcc_vector19 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\4.1 커버_29s_mfcc.npy')
mfcc_vector20 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\4.1 커버 key 3_29s_mfcc.npy')
mfcc_vector21 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\4.1 커버 key -3_29s_mfcc.npy')
mfcc_vector22 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\4.1 커버 noise 1_29s_mfcc.npy')
mfcc_vector23 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\4.1 커버 noise 2_29s_mfcc.npy')
mfcc_vector24 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\4.2 커버_29s_mfcc.npy')

mfcc_vector25 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\5.1 샘플링_29s_mfcc.npy')
mfcc_vector26 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\5.1 샘플링 key 3_29s_mfcc.npy')
mfcc_vector27 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\5.1 샘플링 key -3_29s_mfcc.npy')
mfcc_vector28 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\5.1 샘플링 noise 1_29s_mfcc.npy')
mfcc_vector29 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\5.1 샘플링 noise 2_29s_mfcc.npy')
mfcc_vector30 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\5.2 샘플링_29s_mfcc.npy')

mfcc_vector31 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\6.1 샘플링_29s_mfcc.npy')
mfcc_vector32 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\6.1 샘플링 key 3_29s_mfcc.npy')
mfcc_vector33 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\6.1 샘플링 key -3_29s_mfcc.npy')
mfcc_vector34 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\6.1 샘플링 noise 1_29s_mfcc.npy')
mfcc_vector35 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\6.1 샘플링 noise 2_29s_mfcc.npy')
mfcc_vector36 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\6.2 샘플링_29s_mfcc.npy')

mfcc_vector37 = np.load('C:\\Users\\haim1\\new_folder\\test_mfcc\\7.1 다른곡_29s_mfcc.npy')


for i in range(1, 38):
    globals()[f'mfcc_vector{i}'] =  globals()[f'mfcc_vector{i}'].reshape(1, 13, -1)

In [ ]:
print("Result of MFCC\n")

print('[1.1 vs 1.1 key 3]\유사도:', result([mfcc_vector1, mfcc_vector2]))
print('[1.1 vs 1.1 key -3]\n유사도:', result([mfcc_vector1, mfcc_vector3]))
print('[1.1 vs 1.1 noise 1]\n유사도:', result([mfcc_vector1, mfcc_vector4]))
print('[1.1 vs 1.1 noise 2]\n유사도:', result([mfcc_vector1, mfcc_vector5]))
print('[1.1 vs 다른 노래]\n유사도:', result([mfcc_vector1, mfcc_vector37]))
print('[1.1 vs 1.2]\n유사도:', result([mfcc_vector1, mfcc_vector6]))

print("------------------------------------")

print('[2.1 vs 2.1 key 3]\n유사도:', result([mfcc_vector7, mfcc_vector8]))
print('[2.1 vs 2.1 key -3]\n유사도:', result([mfcc_vector7, mfcc_vector9]))
print('[2.1 vs 2.1 noise 1]\n유사도:', result([mfcc_vector7, mfcc_vector10]))
print('[2.1 vs 2.1 noise 2]\n유사도:', result([mfcc_vector7, mfcc_vector11]))
print('[2.1 vs 다른 노래]\n유사도:', result([mfcc_vector7, mfcc_vector37]))
print('[2.1 vs 2.2]\n유사도:', result([mfcc_vector7, mfcc_vector12]))

print("------------------------------------")

print('[3.1 vs 3.1 key 3]\n유사도:', result([mfcc_vector13, mfcc_vector14]))
print('[3.1 vs 3.1 key -3]\n유사도:',result([mfcc_vector13, mfcc_vector15]))
print('[3.1 vs 3.1 noise 1]\n유사도:', result([mfcc_vector13, mfcc_vector16]))
print('[3.1 vs 3.1 noise 2]\n유사도:', result([mfcc_vector13, mfcc_vector17]))
print('[3.1 vs 다른 노래]\n유사도:', result([mfcc_vector13, mfcc_vector37]))
print('[3.1 vs 3.2]\n유사도:', result([mfcc_vector13, mfcc_vector18]))

print("------------------------------------")

print('[4.1 vs 4.1 key 3]\n유사도:', result([mfcc_vector19, mfcc_vector20]))
print('[4.1 vs 4.1 key -3]\n유사도:', result([mfcc_vector19, mfcc_vector21]))
print('[4.1 vs 4.1 noise 1]\n유사도:', result([mfcc_vector19, mfcc_vector22]))
print('[4.1 vs 4.1 noise 2]\n유사도:', result([mfcc_vector19, mfcc_vector23]))
print('[4.1 vs 다른 노래]\n유사도:', result([mfcc_vector19, mfcc_vector37]))
print('[4.1 vs 4.2]\n유사도:',result([mfcc_vector19, mfcc_vector24]))

print("------------------------------------")

print('[5.1 vs 5.1 key 3]\n유사도:',result([mfcc_vector25, mfcc_vector26]))
print('[5.1 vs 5.1 key -3]\n유사도:', result([mfcc_vector25, mfcc_vector27]))
print('[5.1 vs 5.1 noise 1]\n유사도:', result([mfcc_vector25, mfcc_vector28]))
print('[5.1 vs 5.1 noise 2]\n유사도:', result([mfcc_vector25, mfcc_vector29]))
print('[5.1 vs 다른 노래]\n유사도:', result([mfcc_vector25, mfcc_vector37]))
print('[5.1 vs 5.2]\n유사도:',result([mfcc_vector25, mfcc_vector30]))

print("------------------------------------")

print('[6.1 vs 6.1 key 3]\n유사도:', result([mfcc_vector31, mfcc_vector32]))
print('[6.1 vs 6.1 key -3]\n유사도:',result([mfcc_vector31, mfcc_vector33]))
print('[6.1 vs 6.1 noise 1]\n유사도:', result([mfcc_vector31, mfcc_vector34]))
print('[6.1 vs 6.1 noise 2]\n유사도:', result([mfcc_vector31, mfcc_vector35]))
print('[6.1 vs 다른 노래]\n유사도:', result([mfcc_vector31, mfcc_vector37]))
print('[6.1 vs 6.2]\n유사도:', result([mfcc_vector31, mfcc_vector36]))